# 課題 スクレイピングを業務で活用するアイデアを考えてみよう

[Yahoo! JAPANのトップページ](https://www.yahoo.co.jp/)から主要ニュースのリストを取得し、Markdown形式に変換して表示する。

## LLMを使わない場合

In [2]:
import requests
from bs4 import BeautifulSoup
from dataclasses import dataclass

@dataclass
class News:
  title: str
  url: str

def fetch_news_list() -> list[News]:
  response = requests.get("https://www.yahoo.co.jp/")
  response.raise_for_status()
  soup = BeautifulSoup(response.content, "html.parser")
  news_links = soup.select('#tabpanelTopics1 [aria-label="主要 ニュース"] a[href^="https://news.yahoo.co.jp/"]')
  news_list = []
  
  for link in news_links:
    title = link.select_one('h1').get_text(strip=True)
    url = link.get('href')
    news = News(title=title, url=url)
    news_list.append(news)

  return news_list

def to_markdown(news_list: list[News]) -> str:
  markdown_lines = []

  for news in news_list:
    markdown_lines.append(f"- [{news.title}]({news.url})")

  return "\n".join(markdown_lines)

def fetch_news_list_not_using_llm() -> str:
  news_list = fetch_news_list()
  return to_markdown(news_list)

if __name__ == "__main__":
  markdown_not_using_llm = fetch_news_list_not_using_llm()
  print(markdown_not_using_llm)

- [中継 同友会・新浪代表幹事が会見](https://news.yahoo.co.jp/pickup/6551251)
- [九州太平洋側 3日夜から大雨恐れ](https://news.yahoo.co.jp/pickup/6551243)
- [JAL機長飲酒 国交省立ち入り監査](https://news.yahoo.co.jp/pickup/6551246)
- [わいせつ行為生配信疑い 4人逮捕](https://news.yahoo.co.jp/pickup/6551242)
- [見知らぬ人から尾行 気づくか検証](https://news.yahoo.co.jp/pickup/6551240)
- [温泉休止SNSで拡散 復旧伝わらず](https://news.yahoo.co.jp/pickup/6551228)
- [記憶喪失の男性 有力情報相次ぐ](https://news.yahoo.co.jp/pickup/6551247)
- [清水尋也容疑者を逮捕 事務所謝罪](https://news.yahoo.co.jp/pickup/6551244)


## LLMを使う場合

In [3]:
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
from openai import OpenAI

MODEL_NAME = "gpt-4o-mini"

def fetch_news_list_using_llm() -> str:
  response = requests.get("https://www.yahoo.co.jp/")
  response.raise_for_status()
  soup = BeautifulSoup(response.content, "html.parser")
  body = str(soup.body)
  load_dotenv("../.env")
  client = OpenAI(api_key=os.environ['API_KEY'])
  prompt = f"""
  以下のHTMLから主要ニュースを抽出し、タイトルとURLを以下のようなMarkdown形式で出力してくだい。
  一覧以外（コードブロック用の```など）は出力しないでください。

  出力形式:
  - [タイトル](URL)

  HTML:
  ```html
  {body[:100000]}
  ```
  """
  response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
      {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
  )
  return response.choices[0].message.content.strip()

if __name__ == "__main__":
  markdown_using_llm = fetch_news_list_using_llm()
  print(markdown_using_llm)

- [中継 同友会・新浪代表幹事が会見](https://news.yahoo.co.jp/pickup/6551251)
- [九州太平洋側 3日夜から大雨恐れ](https://news.yahoo.co.jp/pickup/6551243)
- [JAL機長飲酒 国交省立ち入り監査](https://news.yahoo.co.jp/pickup/6551246)
- [わいせつ行為生配信疑い 4人逮捕](https://news.yahoo.co.jp/pickup/6551242)
- [見知らぬ人から尾行 気づくか検証](https://news.yahoo.co.jp/pickup/6551240)
- [温泉休止SNSで拡散 復旧伝わらず](https://news.yahoo.co.jp/pickup/6551228)
- [記憶喪失の男性 有力情報相次ぐ](https://news.yahoo.co.jp/pickup/6551247)
- [清水尋也容疑者を逮捕 事務所謝罪](https://news.yahoo.co.jp/pickup/6551244)


## テスト

In [4]:
assert markdown_not_using_llm == markdown_using_llm